# Rules
- 4 observations a week, 6 during shutdown period
- start on week 2nd 2026 (5th January)
- The same person shouldn't do the same type of observation round twice (i.e., not the same area, not the same ) 
- AT/SJA - every 2 weeks
- Some person e.g., Dorte should be sent to specific more technical topics 
- Persons should have certain skills assigned
- Areas should have certain skill requirements
- AT/SJA should have "Vælg selv relevant område"


In [218]:
import pandas as pd
from datetime import date
import random
from collections import defaultdict

In [219]:
year = 2026
start_week = 2
end_week = 52
obs_week = 4

### Build the framework

In [220]:
def skeleton(year, start_week, end_week, obs_week):
    rows = []
    for w in range(start_week, end_week + 1):
        monday = date.fromisocalendar(year, w, 1)
        for slot in range(1, obs_week + 1):
            rows.append({
            "Week": f"W{w:02d}{year}",
            "Week_Monday": monday,
            "Slot": slot,
            "Person": None,
            "Area": None,
            "Companion": None,
            "Topic": None,
            })
    return pd.DataFrame(rows)

In [221]:
if __name__ == "__main__":
    df = skeleton(year, start_week, end_week, obs_week)
    print(df.head(10))
    print("Total rows:", len(df))

      Week Week_Monday  Slot Person  Area Companion Topic
0  W022026  2026-01-05     1   None  None      None  None
1  W022026  2026-01-05     2   None  None      None  None
2  W022026  2026-01-05     3   None  None      None  None
3  W022026  2026-01-05     4   None  None      None  None
4  W032026  2026-01-12     1   None  None      None  None
5  W032026  2026-01-12     2   None  None      None  None
6  W032026  2026-01-12     3   None  None      None  None
7  W032026  2026-01-12     4   None  None      None  None
8  W042026  2026-01-19     1   None  None      None  None
9  W042026  2026-01-19     2   None  None      None  None
Total rows: 204


In [222]:
def add_extra_slots(df, weeks=(16,17), total_slots=6):
    out = df.copy()
    for w in weeks:
        mask = out["Week_Monday"].apply(lambda d: d.isocalendar()[1] == w)
        if not mask.any():
            print(f"[add_extra_slots] Week {w} not found"); 
            continue

        monday = out.loc[mask, "Week_Monday"].iloc[0]
        wk_str = out.loc[mask, "Week"].iloc[0]
        cur_max = int(out.loc[mask, "Slot"].max())  # current highest slot (should be 4)

        added = 0
        for s in range(cur_max + 1, total_slots + 1):
            out.loc[len(out)] = {
                "Week": wk_str, "Week_Monday": monday, "Slot": s,
                "Person": None, "Area": None, "Companion": None, "Topic": None,
            }
            added += 1
        print(f"[add_extra_slots] Week {w}: added {added} slots (now up to {total_slots})")
    return out.sort_values(["Week_Monday", "Slot"]).reset_index(drop=True)


df = add_extra_slots(df, weeks=(16,17), total_slots=6)

for w in (16, 17):
    m = df["Week_Monday"].apply(lambda d: d.isocalendar()[1] == w)
    print("week", w, "| rows:", int(m.sum()), "| slots:", df.loc[m, "Slot"].tolist())


[add_extra_slots] Week 16: added 2 slots (now up to 6)
[add_extra_slots] Week 17: added 2 slots (now up to 6)
week 16 | rows: 6 | slots: [1, 2, 3, 4, 5, 6]
week 17 | rows: 6 | slots: [1, 2, 3, 4, 5, 6]


### Define Categories

In [223]:
PROC = {"process"}
SAFE = {"safety"}
PEOP = {"people"}
ANY  = set()  # means "no specific skill required"
REQ_ANY_SKILL  = [PROC, SAFE, PEOP]
REQ_SAFE_OR_PEOP = [SAFE, PEOP]
REQ_PROC_OR_SAFE = [PROC, SAFE]


def person_skills(*keys):
    m = {"PROC": PROC, "SAFE": SAFE, "PEOP": PEOP, "ALL": ALL_SKILLS}
    out = set()
    for k in keys:
        out |= m[k]
    return out


In [224]:
Persons_sk = {
    "Casper B. Hansen": SAFE,
    "Maria Bjerregaard": person_skills("PROC", "SAFE"), 
    "Kent Carlsen": ALL_SKILLS,
    "Thomas M. Dietz": SAFE,
    "Peter H. Madsen": SAFE,
    "Kris L. B. Mikkelsen": SAFE,
    "Stig Mikkelsen": SAFE,
    "Janne Krogager": PROC,
    "Richardt Brixen": ALL_SKILLS,
    "Alison Sørensen": PEOP,
    "Allan Krestensen": PEOP,
    "Niels Bech": PROC,
    "Jens Ruskol": person_skills("PROC", "SAFE"),       
    "Rikke Knudsen": ALL_SKILLS,
    "Tanja B. Jensen": SAFE,
    "Bo Neville": PEOP,
    "Michael L. Jensen": PEOP,
    "Rene Feldskov": SAFE,
    "Rasmus H. H. Klausen": PROC,
    "Patrik Kjellander": SAFE,
    "Robert S. Axelsen": ALL_SKILLS,
    "Karsten Eliasen": SAFE,
    "Denis Ordell Skovhus": SAFE,
    "Niki Hampe-Lander": PEOP,
    "Niels B. Trelleborg": PEOP,
    "Dorte Larsen": PROC,
    "Hans J. Christiansen": SAFE,
    "Jimmi Nielsen": SAFE,
    "Ulrik N. Olsen": SAFE,
    "Louise K. Dalgaard": PEOP,
    "Lars Eliasson": ALL_SKILLS,
    "Steffen Pedersen": SAFE,
    "Michael Elversøe": SAFE,
    "Ole D. Olsen": SAFE,
    "Søren Værnstrøm": SAFE,
    "Søren G. Hansen": ALL_SKILLS,
    "Lars L. Jensen": person_skills("PROC", "SAFE"),    
    "Morten Mølgaard": PEOP,
    "Annette Munch": ALL_SKILLS,
    "Jan Therkelsen": SAFE,
    "Stig Fruehøj": ALL_SKILLS,
    "Rasmus Møller": PEOP,
    "Jamie Uniacke": PROC,
    "Claus Navntoft": ALL_SKILLS,
    "Janne B. Saxtoft": SAFE,
    "Sara B. Sandersen": PROC,
}

for p, v in list(Persons_sk.items()):
    if isinstance(v, list) and all(isinstance(s, set) for s in v):
        Persons_sk[p] = set().union(*v)
    elif not isinstance(v, set):
        Persons_sk[p] = set(v) 


Persons = list(Persons_sk)

In [225]:
Area_requires = {
    "PIER": SAFE,
    "ASV Værksted": SAFE,
    "NCC": PEOP,
    "BLOK 4 og 5": REQ_ANY_SKILL,
    "BLOK 3": REQ_ANY_SKILL,
    "BLOK 2": REQ_ANY_SKILL,
    "BLOK 1": REQ_ANY_SKILL,
    "Administrationsbygningen": PEOP,
    "Hedehusene": PEOP,
    "Warehouse": SAFE,
    "Visbreaker": REQ_ANY_SKILL,
    "TA-bygninger": PEOP,
    "Omkring hovedport og vestomklædningen": SAFE,
    "Omkring østport": SAFE,
    "Off-Site": REQ_ANY_SKILL,
    "BRB": PEOP,
    "Vælg selv relevant område": ANY,   
}


Area = list(Area_requires)

Area_weights = {
    "PIER": 1.5,
    "ASV Værksted": 1.0,
    "NCC" : 1.0,
    "BLOK 4 og 5" : 1.5,
    "BLOK 3" : 1.5,
    "BLOK 2" : 1.5,
    "BLOK 1" : 1.5,
    "Administrationsbygningen" : 1.0,
    "Hedehusene" : 0.1,
    "Warehouse" : 0.5,
    "Visbreaker" : 1.5,
    "TA-bygninger" : 1.0,
    "Omkring hovedport og vestomklædningen": 1.0,
    "Omkring østport": 1.0,
    "Off-Site": 1.5,
    "BRB": 1.0,
    "Vælg selv relevant område": 0.0,

}


In [226]:
Topic_requires = {
    "Adfærd/sikkerhedsdialog": REQ_SAFE_OR_PEOP,
    "RCA opfølgning på afsluttet RCA": REQ_PROC_OR_SAFE,  
    "Arbejde i højden/arbejde i confined spaces": SAFE,
    "Process sikkerhed": PROC,
    "Risikovurdering /TaTo/Før-job-samtale": SAFE,
    "Adgangsvej/flugtvej/orden og ryddelighed/færdsel/parkering": REQ_SAFE_OR_PEOP,  
    "Kemikalier/brugsanvisninger/værnemidler/miljø og affaldssortering": REQ_PROC_OR_SAFE,  
    "Løfteudstyr/stillads/afspærring/afdækning": SAFE,
    "Kontor/støj/lys/træk/ryddelighed": PEOP,
    "AT/SJA": REQ_PROC_OR_SAFE,
}

Topic = list(Topic_requires)

Topic_weights = {
"Adfærd/sikkerhedsdialog": 1.5,
"RCA opfølgning på afsluttet RCA": 0.5,
"Arbejde i højden/arbejde i confined spaces": 1.0,
"Process sikkerhed": 1.0,
"Risikovurdering /TaTo/Før-job-samtale": 1.5,
"Adgangsvej/flugtvej/orden og ryddelighed/færdsel/parkering": 1.5,
"Kemikalier/brugsanvisninger/værnemidler/miljø og affaldssortering": 0.8,
"Løfteudstyr/stillads/afspærring/afdækning": 1.0,
"Kontor/støj/lys/træk/ryddelighed": 1.0,
"AT/SJA": 0.0,
}

In [227]:
Companion =["Kollega fra HSEQ inkl brandstation", "Vælg selv relevant kollega", "Kollega - ikke egen afdeling"]

### Define Rules

In [228]:
def assign_persons(df, persons):
    df = df.sort_values(["Week_Monday", "Slot"]).copy()
    df["Person"] = [persons[i % len(persons)] for i in range(len(df))]
    return df

df = assign_persons(df, Persons) 
df.head(8)

,Week,Week_Monday,Slot,Person,Area,Companion,Topic
0,W022026,2026-01-05,1,Casper B. Hansen,None,None,None
1,W022026,2026-01-05,2,Maria Bjerregaard,None,None,None
2,W022026,2026-01-05,3,Kent Carlsen,None,None,None
3,W022026,2026-01-05,4,Thomas M. Dietz,None,None,None
4,W032026,2026-01-12,1,Peter H. Madsen,None,None,None
5,W032026,2026-01-12,2,Kris L. B. Mikkelsen,None,None,None
6,W032026,2026-01-12,3,Stig Mikkelsen,None,None,None
7,W032026,2026-01-12,4,Janne Krogager,None,None,None


In [229]:
def assign_persons(df, persons):
    df = df.sort_values(["Week_Monday", "Slot"]).copy()
    df["Person"] = [persons[i % len(persons)] for i in range(len(df))]
    return df

df = assign_persons(df, Persons) 
df.head(8)

,Week,Week_Monday,Slot,Person,Area,Companion,Topic
0,W022026,2026-01-05,1,Casper B. Hansen,None,None,None
1,W022026,2026-01-05,2,Maria Bjerregaard,None,None,None
2,W022026,2026-01-05,3,Kent Carlsen,None,None,None
3,W022026,2026-01-05,4,Thomas M. Dietz,None,None,None
4,W032026,2026-01-12,1,Peter H. Madsen,None,None,None
5,W032026,2026-01-12,2,Kris L. B. Mikkelsen,None,None,None
6,W032026,2026-01-12,3,Stig Mikkelsen,None,None,None
7,W032026,2026-01-12,4,Janne Krogager,None,None,None


In [230]:
def _eligible(req, skills):
    if isinstance(req, list):      # OR
        return any(r.issubset(skills) for r in req)
    return req.issubset(skills)  

In [231]:
def eligible_for_area(person: str, area: str) -> bool:
    req    = Area_requires.get(area, set())
    skills = Persons_sk.get(person, set())
    return _eligible(req, skills)

In [232]:
def eligible_for_topic(person: str, topic: str) -> bool:
    req    = Topic_requires.get(topic, set())
    skills = Persons_sk.get(person, set())
    return _eligible(req, skills)

In [233]:
def at_sja(df, start_week=2, every=2, at_topic="AT/SJA"):
    out = df.sort_values(["Week_Monday", "Slot"]).copy()
    for monday, g in out.groupby("Week_Monday"):
        week_no = monday.isocalendar()[1]
        if (week_no - start_week) % every != 0:
            continue

        candidates = []
        for i, row in g.sort_values("Slot").iterrows():
            p = row["Person"]
            if eligible_for_topic(p, at_topic):
                candidates.append((i, p))

        if not candidates:
            continue

        # prefer a person who hasn't had AT/SJA yet
        pick = None
        for i, p in candidates:
            if at_topic not in used_topic_person[p]:
                pick = (i, p); break
        if pick is None:
            pick = candidates[0]

        i, p = pick
        out.at[i, "Topic"] = at_topic
        used_topic_person[p].add(at_topic)
    return out

### Add Weights

In [234]:
def _weighted_choice(options, weight_dict, rng):   
    w = [weight_dict.get(x, 1.0) for x in options]
    s = sum(w)
    if s <= 0:
        return rng.choice(options)
    r = rng.random() * s
    acc = 0.0
    for x, wi in zip(options, w):
        acc += wi
        if r <= acc:
            return x
    return options[-1]

In [235]:
used_topic_person = defaultdict(set)

def fill_remaining_topics(df, topics, topic_requires, topic_weights, persons_sk, seed=42, at_topic="AT/SJA"):
    import pandas as pd, random
    rng = random.Random(seed)
    out = df.sort_values(["Week_Monday", "Slot"]).copy()

    for monday, idx in out.groupby("Week_Monday").groups.items():
        used_week = set(out.loc[idx, "Topic"].dropna())  # topics already set this week (e.g., AT/SJA)

        for i in out.loc[idx].sort_values("Slot").index:
           
            if pd.notna(out.at[i, "Topic"]):
                continue

            person = out.at[i, "Person"]

            # prefer topics the PERSON hasn't done yet (no repeat per person), no weekly repeat, exclude AT/SJA
            pool = [t for t in topics
                    if t != at_topic
                    and t not in used_week
                    and eligible_for_topic(person, t)
                    and t not in used_topic_person[person]]

            # fallback: allow repeat for the person if no new eligible topic fits (still no weekly repeat)
            if not pool:
                pool = [t for t in topics
                        if t != at_topic
                        and t not in used_week
                        and eligible_for_topic(person, t)]

            if not pool:
                out.at[i, "Topic"] = "UNASSIGNED"
                continue

            choice = _weighted_choice(pool, topic_weights, rng)
            out.at[i, "Topic"] = choice
            used_week.add(choice)
            used_topic_person[person].add(choice)

    return out

df = at_sja(df, start_week=start_week, every=2, at_topic="AT/SJA")
df = fill_remaining_topics(df, Topic, Topic_requires, Topic_weights, Persons_sk)

In [236]:
used_area = {p: set() for p in Persons}   
used_area_week = defaultdict(set)         
week_area_count = defaultdict(int)      
_rng = random.Random(42)   

#def assign_areas(df):
 #   out = df.sort_values(["Week_Monday","Slot"]).copy()
  #  for i, row in out.iterrows():
   #     person = row["Person"]
    #    week_monday = row["Week_Monday"]
  #      week_key = (week_monday, person)

     
   #     elig = [a for a in Area
     #           if Area_requires.get(a, set()).issubset(Persons_sk.get(person, set()))]
     #   if not elig:
 #           out.at[i, "Area"] = "Vælg selv relevant område"; continue

       
  #      not_this_week = [a for a in elig if a not in used_area_week[week_key]]
  #      if not not_this_week:
  #          out.at[i, "Area"] = "Vælg selv relevant område"; continue

     
 #       not_taken_yet = [a for a in not_this_week if week_area_count[(week_monday, a)] < 1]
 #       if not not_taken_yet:
#            out.at[i, "Area"] = "Vælg selv relevant område"; continue

       
#        pool = [a for a in not_taken_yet if a not in used_area[person]]
 #       if not pool:
  #          used_area[person].clear()
  #          pool = not_taken_yet

 #       choice = _weighted_choice(pool, Area_weights, _rng)
  #      out.at[i, "Area"] = choice
 #       used_area[person].add(choice)
 #       used_area_week[week_key].add(choice)
 #       week_area_count[(week_monday, choice)] += 1
 #   return out

def assign_areas(df, at_topic="AT/SJA", at_area="Vælg selv relevant område"):
    out = df.sort_values(["Week_Monday", "Slot"]).copy()

    # Iterate week-by-week so we can reserve the AT/SJA area first
    for monday, idx in out.groupby("Week_Monday").groups.items():
        week_rows = out.loc[idx].sort_values("Slot")

        # --- PASS 1: reserve area for AT/SJA if present this week ---
        at_idx = None
        for i in week_rows.index:
            if out.at[i, "Topic"] == at_topic:
                at_idx = i
                person = out.at[i, "Person"]
                week_key = (monday, person)

                # force the reserved area
                out.at[i, "Area"] = at_area
                used_area[person].add(at_area)
                used_area_week[week_key].add(at_area)
                week_area_count[(monday, at_area)] += 1
                break  # only one AT/SJA per week with your current logic

        # --- PASS 2: assign areas for all other rows in the week ---
        for i in week_rows.index:
            # skip if area already set (e.g., AT/SJA we just forced)
            if pd.notna(out.at[i, "Area"]) and out.at[i, "Area"] is not None:
                continue

            person = out.at[i, "Person"]
            week_key = (monday, person)

            # 1) eligibility gate
            elig = [a for a in Area
                    if Area_requires.get(a, set()).issubset(Persons_sk.get(person, set()))]
            if not elig:
                out.at[i, "Area"] = "Vælg selv relevant område"; continue

            # 2) not repeated for this person in this week
            not_this_week = [a for a in elig if a not in used_area_week[week_key]]
            if not not_this_week:
                out.at[i, "Area"] = "Vælg selv relevant område"; continue

            # 3) only once per area per week (respect reservation we just did)
            not_taken_yet = [a for a in not_this_week if week_area_count[(monday, a)] < 1]
            if not not_taken_yet:
                out.at[i, "Area"] = "Vælg selv relevant område"; continue

            # 4) avoid repeating for the person until cycle completes
            pool = [a for a in not_taken_yet if a not in used_area[person]]
            if not pool:
                used_area[person].clear()
                pool = not_taken_yet

            # 5) pick with weights
            choice = _weighted_choice(pool, Area_weights, _rng)
            out.at[i, "Area"] = choice
            used_area[person].add(choice)
            used_area_week[week_key].add(choice)
            week_area_count[(monday, choice)] += 1

    return out


df = assign_areas(df)
df.head(10)[["Week","Slot","Person","Area","Topic"]]

AttributeError: 'list' object has no attribute 'issubset'

In [ ]:
used_comp = {p: set() for p in Persons}      
used_comp_week = defaultdict(set)              
_rng = random.Random(42)

def assign_companions(df, companions, avoid_self=True):
    out = df.sort_values(["Week_Monday","Slot"]).copy()
    for i, row in out.iterrows():
        person = row["Person"]
        week_key = (row["Week_Monday"], person)

        # 1) eligible companions (optionally forbid self)
        elig = [c for c in companions if not (avoid_self and c == person)]
        if not elig:
            out.at[i, "Companion"] = "UNASSIGNED"; continue

        # 2) block repeats for THIS person in THIS week
        not_this_week = [c for c in elig if c not in used_comp_week[week_key]]
        if not not_this_week:
            out.at[i, "Companion"] = "UNASSIGNED"; continue

        # 3) avoid repeating until cycle completes
        pool = [c for c in not_this_week if c not in used_comp[person]]
        if not pool:
            used_comp[person].clear()
            pool = not_this_week

        # pick (uniform; pass weights via _weighted_choice if you want bias)
        choice = _weighted_choice(pool, {}, _rng)
        out.at[i, "Companion"] = choice
        used_comp[person].add(choice)
        used_comp_week[week_key].add(choice)
    return out


df = assign_companions(df, Companion, avoid_self=False)  # set True if names can match persons
df.head(10)[["Week","Slot","Person","Companion", "Topic"]]


### Export to excel

In [ ]:
cols = ["Week","Week_Monday","Slot","Person","Area","Companion","Topic"]
df_final = df.sort_values(["Week_Monday","Slot"])[cols].reset_index(drop=True)

XLSX_OUT = r"C:/Users/ALMU/OneDrive - Kalundborg Refinery/Documents/01. Projects/02. Data Analysis/00. Python_Data Exported/obs_rounds_2026.xlsx"
df_final.to_excel(XLSX_OUT, index=False)


### Debugging

In [ ]:
print("Casper:", eligible_for_topic("Casper B. Hansen","Adfærd/sikkerhedsdialog"))
print("Michael:", eligible_for_topic("Michael L. Jensen","Adfærd/sikkerhedsdialog"))